### Training monitoring

In [2]:
import torch
from torchvision import transforms
import gc

from ClassComp.models.vgg import VGG
from ClassComp.models.resnet import ResNet
from ClassComp.models.unet import UNet
from ClassComp.models.vae import VAE, SVMLoss, VAE_conv
from ClassComp.data_utils.loaders import get_mnist, get_dataloader
from ClassComp.experiments.train import train_binary_classifier, train_vae, train_vae_kmeans

epochs = 10 # can go up to  20 but will be very slow
image_size = 64
batch_size = 16
custom_transforms = None

learning_rates = [1e-3]

## Get binary MNIST subsets
train_subset, test_subset = get_mnist()

## Get DataLoaders with additional transformations
train_loader, test_loader = get_dataloader(train_subset, test_subset, transform=custom_transforms, size=image_size, batch_size=batch_size)



def del_model(model):
    del model
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.empty_cache()
    gc.collect()



In [5]:

# Vanilla training

description = "vanilla"

## Train
for i, lr in enumerate(learning_rates):
    print("Training VGG")
    vgg = VGG(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(vgg, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vgg.state_dict(), f"./results/models/vgg_{epochs}_epochs_{lr}_lr_{description}.pth.tar")

    del_model(vgg)

    print("Training Resnet")
    resnet = ResNet(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(resnet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        resnet_path = f"./results/models/resnet_{epochs}_epochs_{lr}_lr_{description}.pth.tar"
        torch.save(resnet.state_dict(), resnet_path)

    del_model(resnet)

    print("Training Unet")
    for j in range(2):
        if j == 0:
            resnet = ResNet(input_img_size=image_size, input_img_c=1)  
            unet_description = description + "_pretrained_resnet"
            resnet.load_state_dict(torch.load(resnet_path))
            resnet.eval()
            resnet.to("cuda")
            unet = UNet(image_size, resnet)
        else:
            unet_description = description + "no_pretraining"
            unet = UNet(image_size)
        
        
        train_binary_classifier(unet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=unet_description)

        if i == 0:
            torch.save(unet.state_dict(), f"./results/models/unet_{epochs}_epochs_{lr}_lr_{unet_description}.pth.tar")

        del_model(unet)

    print("Training VAE")
    # for j in range(2):
    vae = VAE(image_size**2, 32*32, 8, beta=0.05)
        # if j == 0:
        #     resnet = ResNet(input_img_size=image_size, input_img_c=1)  
        #     resnet.load_state_dict(torch.load(resnet_path))
        #     resnet.eval()
        #     resnet.to("cuda")
        #     vae = VAE_conv(image_size, resnet)
        #     vae_description = description + "_pretrained_resnet"
        # else:
        #     vae_description = description + "no_pretraining"
        #     vae = VAE_conv(image_size)

    train_vae(vae, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_generation.pth.tar")

    vae.classification_mode = "SVM"

    # Train VAE classification head
    criterion = SVMLoss()
    for param in vae.parameters():
        param.requires_grad = False
    for name, param in vae.named_parameters():
        if "svm_layer" in name:
            param.requires_grad = True
    train_binary_classifier(vae, train_loader, test_loader, criterion=criterion, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_classification.pth.tar")

    train_vae_kmeans(vae, train_loader, test_loader, "cuda", True, description)
    
    del_model(vae)



Training VAE


/tmp/ipykernel_821108/3003024826.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(f"./results/models/vae_{epochs}_epochs_{lr}_lr_{descript

0.9446808510638298
Training metrics saved to results/training/VAE_vanilla_kmeans.pkl
Kmeans saved to results/models/VAE_vanilla_kmeans.pkl
Training VAE


/tmp/ipykernel_821108/3003024826.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(f"./results/models/vae_{epochs}_epochs_{lr}_lr_{descript

FileNotFoundError: [Errno 2] No such file or directory: './results/models/vae_10_epochs_1e-05_lr_vanilla_generation.pth.tar'

In [ ]:
# Noisy training

image_size = 64
batch_size = 16

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)
    
custom_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of horizontal flip
    transforms.RandomVerticalFlip(p=0.5),    # 50% chance of vertical flip
    transforms.RandomAffine(
        degrees=30,                          # Random rotation within ±30 degrees
        translate=(0.1, 0.1),                # Random translation up to 10% of image size
        scale=(0.9, 1.1),                    # Random scaling between 90% and 110%
        shear=10                             # Random shear within ±10 degrees
    ),
    transforms.RandomPerspective(
        distortion_scale=0.5,                # Distortion scale for perspective transform
        p=0.5                                # 50% chance of applying
    ),
    transforms.RandomErasing(
        p=0.5,                               # 50% chance of applying
        scale=(0.02, 0.2),                   # Proportion of erased area
        ratio=(0.3, 3.3),                    # Aspect ratio of erased area
        value=0                              # Fill value for erased pixels
    ),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),  # Add Gaussian Blur
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),  # Adjust sharpness
    AddGaussianNoise(),
])

learning_rates = [1e-3, 1e-5]
description = "class_imbalance_noisy"

## Get binary MNIST subsets
train_subset, test_subset = get_mnist()

## Get DataLoaders with additional transformations
train_loader, test_loader = get_dataloader(train_subset, test_subset, transform=custom_transforms, size=image_size, batch_size=batch_size, class_imbalance=0.6)

## Train
for i, lr in enumerate(learning_rates):
    print("Training VGG")
    vgg = VGG(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(vgg, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vgg.state_dict(), f"./results/models/vgg_{epochs}_epochs_{lr}_lr_{description}.pth.tar")

    del_model(vgg)

    print("Training Resnet")
    resnet = ResNet(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(resnet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        resnet_path = f"./results/models/resnet_{epochs}_epochs_{lr}_lr_{description}.pth.tar"
        torch.save(resnet.state_dict(), resnet_path)

    del_model(resnet)

    print("Training Unet")
    for j in range(2):
        if j == 0:
            resnet = ResNet(input_img_size=image_size, input_img_c=1)  
            unet_description = description + "_pretrained_resnet"
            resnet.load_state_dict(torch.load(resnet_path))
            resnet.eval()
            resnet.to("cuda")
            unet = UNet(image_size, resnet)
        else:
            unet_description = description + "no_pretraining"
            unet = UNet(image_size)

        train_binary_classifier(unet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=unet_description)

        if i == 0:
            torch.save(unet.state_dict(), f"./results/models/unet_{epochs}_epochs_{lr}_lr_{unet_description}.pth.tar")

        del_model(unet)

    print("Training VAE")
    # for j in range(2):
    vae = VAE(image_size**2, 32*32, 8, beta=0.05)
        

    train_vae(vae, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_generation.pth.tar")

    vae.classification_mode = "SVM"

    # Train VAE classification head
    criterion = SVMLoss()
    for param in vae.parameters():
        param.requires_grad = False
    for name, param in vae.named_parameters():
        if "svm_layer" in name:
            param.requires_grad = True
    train_binary_classifier(vae, train_loader, test_loader, criterion=criterion, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_classification.pth.tar")

    train_vae_kmeans(vae, train_loader, test_loader, "cuda", True, description)


    del_model(vae)


Training VGG


Epoch [1/20], Loss: 0.6649, Training Accuracy: 59.14%, Time: 89.74s, Gradient Norm: 5.7491
Validation Accuracy: 53.66%


Epoch [2/20], Loss: 0.6199, Training Accuracy: 60.53%, Time: 87.08s, Gradient Norm: 2.5078
Validation Accuracy: 53.66%


Epoch [3/20], Loss: 0.6073, Training Accuracy: 61.37%, Time: 83.01s, Gradient Norm: 2.2063
Validation Accuracy: 75.84%


Epoch [4/20], Loss: 0.3985, Training Accuracy: 84.05%, Time: 84.22s, Gradient Norm: 3.8266
Validation Accuracy: 85.77%


...

Epoch [16/20], Loss: 0.6622, Training Accuracy: 46.62%, Time: 54.13s, Gradient Norm: 2.4322
Validation Accuracy: 44.02%


Epoch [17/20], Loss: 0.6577, Training Accuracy: 47.84%, Time: 57.51s, Gradient Norm: 2.4355
Validation Accuracy: 43.50%


Epoch [18/20], Loss: 0.6484, Training Accuracy: 47.82%, Time: 60.72s, Gradient Norm: 2.4378
Validation Accuracy: 44.54%


Epoch [19/20], Loss: 0.6470, Training Accuracy: 47.63%, Time: 58.83s, Gradient Norm: 2.4177
Validation Accuracy: 43.74%


Epoch [20/20], Loss: 0.6448, Training Accuracy: 47.73%, Time: 59.89s, Gradient Norm: 2.4112
Validation Accuracy: 42.36%
Training metrics saved to results/training/VAE_20_epochs_1e-05_lr_class_imbalance_noisy.pkl
0.45673758865248226
Training metrics saved to results/training/VAE_class_imbalance_noisy_kmeans.pkl
Kmeans saved to results/models/VAE_class_imbalance_noisy_kmeans.pkl


In [15]:
# Strong class imbalance

image_size = 64
batch_size = 16

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)
    
custom_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of horizontal flip
    transforms.RandomVerticalFlip(p=0.5),    # 50% chance of vertical flip
    transforms.RandomAffine(
        degrees=30,                          # Random rotation within ±30 degrees
        translate=(0.1, 0.1),                # Random translation up to 10% of image size
        scale=(0.9, 1.1),                    # Random scaling between 90% and 110%
        shear=10                             # Random shear within ±10 degrees
    ),
    transforms.RandomPerspective(
        distortion_scale=0.5,                # Distortion scale for perspective transform
        p=0.5                                # 50% chance of applying
    ),
    transforms.RandomErasing(
        p=0.5,                               # 50% chance of applying
        scale=(0.02, 0.2),                   # Proportion of erased area
        ratio=(0.3, 3.3),                    # Aspect ratio of erased area
        value=0                              # Fill value for erased pixels
    ),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),  # Add Gaussian Blur
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),  # Adjust sharpness
    AddGaussianNoise(),
])

learning_rates = [1e-3, 1e-5]
description = "class_imbalance"

## Get binary MNIST subsets
train_subset, test_subset = get_mnist()

## Get DataLoaders with additional transformations
train_loader, test_loader = get_dataloader(train_subset, test_subset, transform=None, size=image_size, batch_size=batch_size, class_imbalance=0.9)

## Train
for i, lr in enumerate(learning_rates):
    print("Training VGG")
    vgg = VGG(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(vgg, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vgg.state_dict(), f"./results/models/vgg_{epochs}_epochs_{lr}_lr_{description}.pth.tar")

    del_model(vgg)

    print("Training Resnet")
    resnet = ResNet(input_img_size=image_size, input_img_c=1)
    train_binary_classifier(resnet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        resnet_path = f"./results/models/resnet_{epochs}_epochs_{lr}_lr_{description}.pth.tar"
        torch.save(resnet.state_dict(), resnet_path)

    del_model(resnet)

    print("Training Unet")
    for j in range(2):
        if j == 0:
            resnet = ResNet(input_img_size=image_size, input_img_c=1)  
            unet_description = description + "_pretrained_resnet"
            resnet.load_state_dict(torch.load(resnet_path))
            resnet.eval()
            resnet.to("cuda")
            unet = UNet(image_size, resnet)
        else:
            unet_description = description + "no_pretraining"
            unet = UNet(image_size)

        train_binary_classifier(unet, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=unet_description)

        if i == 0:
            torch.save(unet.state_dict(), f"./results/models/unet_{epochs}_epochs_{lr}_lr_{unet_description}.pth.tar")

        del_model(unet)

    print("Training VAE")
    # for j in range(2):
    vae = VAE(image_size**2, 32*32, 8, beta=0.05)
        

    train_vae(vae, train_loader, test_loader, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_generation.pth.tar")

    vae.classification_mode = "SVM"

    # Train VAE classification head
    criterion = SVMLoss()
    for param in vae.parameters():
        param.requires_grad = False
    for name, param in vae.named_parameters():
        if "svm_layer" in name:
            param.requires_grad = True
    train_binary_classifier(vae, train_loader, test_loader, criterion=criterion, epochs=epochs, learning_rate=lr, device="cuda", save_results=True, description=description)

    if i == 0:
        torch.save(vae.state_dict(), f"./results/models/vae_{epochs}_epochs_{lr}_lr_{description}_classification.pth.tar")

    train_vae_kmeans(vae, train_loader, test_loader, "cuda", True, description)


    del_model(vae)


Training VGG


Epoch [1/20], Loss: 0.3953, Training Accuracy: 97.98%, Time: 79.65s, Gradient Norm: 12.6666
Validation Accuracy: 81.09%


Epoch [2/20], Loss: 0.1061, Training Accuracy: 98.07%, Time: 79.60s, Gradient Norm: 3.9091
Validation Accuracy: 99.57%


Epoch [3/20], Loss: 0.0545, Training Accuracy: 99.01%, Time: 79.50s, Gradient Norm: 6.7300
Validation Accuracy: 99.10%


Epoch [4/20], Loss: 0.0300, Training Accuracy: 99.02%, Time: 78.98s, Gradient Norm: 0.9624
Validation Accuracy: 99.72%


...

Epoch [15/20], Loss: 0.1240, Training Accuracy: 99.94%, Time: 69.08s, Gradient Norm: 2.1489
Validation Accuracy: 100.00%


Epoch [16/20], Loss: 0.0183, Training Accuracy: 99.97%, Time: 67.62s, Gradient Norm: 0.9534
Validation Accuracy: 99.91%


Epoch [17/20], Loss: 0.0000, Training Accuracy: 100.00%, Time: 67.69s, Gradient Norm: 0.0000
Validation Accuracy: 99.86%


Epoch [18/20], Loss: 0.1425, Training Accuracy: 99.95%, Time: 67.63s, Gradient Norm: 2.4028
Validation Accuracy: 99.95%


KeyboardInterrupt: 